[Kubernetes IN Docker](https://github.com/kubernetes-sigs/kind)

[Kubeadm](https://kubernetes.io/docs/reference/setup-tools/kubeadm/)

---
[Installing kubeadm](https://kubernetes.io/docs/setup/production-environment/tools/kubeadm/install-kubeadm/)

In [ ]:
curl -fsSL https://pkgs.k8s.io/core:/stable:/v1.29/deb/Release.key | sudo gpg --dearmor -o /etc/apt/keyrings/kubernetes-apt-keyring.gpg
echo 'deb [signed-by=/etc/apt/keyrings/kubernetes-apt-keyring.gpg] https://pkgs.k8s.io/core:/stable:/v1.29/deb/ /' | sudo tee /etc/apt/sources.list.d/kubernetes.list
sudo apt-get update
sudo apt-get install -y kubelet kubeadm kubectl
sudo apt-mark hold kubelet kubeadm kubectl

---
on control-plane node

In [ ]:
sudo swapoff -a

In [ ]:
sudo swapon --show

In [ ]:
sudo cat << EOF | sudo tee /etc/sysctl.d/k8s.conf
vm.swappiness=0
EOF

In [ ]:
sudo sysctl -p /etc/sysctl.d/k8s.conf

In [ ]:
sudo systemctl restart kubelet

In [ ]:
systemctl status kubelet

---
[Creating a cluster with kubeadm](https://kubernetes.io/docs/setup/production-environment/tools/kubeadm/create-cluster-kubeadm/)

In [ ]:
sudo kubeadm init --control-plane-endpoint=jhub.dtype.info:6443 --apiserver-advertise-address=10.8.0.1 --pod-network-cidr=10.244.0.0/16

In [ ]:
mkdir -p ~/.kube
sudo cp -f /etc/kubernetes/admin.conf ~/.kube/config
sudo chown $(id -u):$(id -g) ~/.kube/config

In [ ]:
kubectl cluster-info dump | grep -E 'cluster-cidr|service-cluster-ip-range'

                            "--service-cluster-ip-range=10.96.0.0/12",
                            "--cluster-cidr=10.244.0.0/16",
                            "--service-cluster-ip-range=10.96.0.0/12",


---
cni

[Installing containerd](https://github.com/containerd/containerd/blob/main/docs/getting-started.md)

In [ ]:
wget https://github.com/containernetworking/plugins/releases/download/v1.4.1/cni-plugins-linux-amd64-v1.4.1.tgz

In [ ]:
tar -tvzf cni-plugins-linux-amd64-v1.4.1.tgz

In [ ]:
mkdir -p ~/containerd-plugins

In [ ]:
tar -xvzf cni-plugins-linux-amd64-v1.4.1.tgz -C ~/containerd-plugins/bin

In [ ]:
sudo cp -r ~/containerd-plugins/bin /opt/cni

In [ ]:
sudo cat << EOF | sudo tee /etc/cni/net.d/10-containerd-net.conflist
{
  "cniVersion": "1.0.0",
  "name": "containerd-net",
  "plugins": [
    {
      "type": "bridge",
      "bridge": "cni0",
      "isGateway": true,
      "ipMasq": true,
      "promiscMode": true,
      "ipam": {
        "type": "host-local",
        "ranges": [
          [{
            "subnet": "10.88.0.0/16"
          }],
          [{
            "subnet": "2001:4860:4860::/64"
          }]
        ],
        "routes": [
          { "dst": "0.0.0.0/0" },
          { "dst": "::/0" }
        ]
      }
    },
    {
      "type": "portmap",
      "capabilities": {"portMappings": true}
    }
  ]
}
EOF

In [ ]:
sudo systemctl restart containerd

---
calico

[Install Calico on Kubernetes](https://docs.tigera.io/calico/latest/getting-started/kubernetes/quickstart)

---
flannel

``` yaml
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: kube-flannel-ds
  namespace: kube-flannel
  labels:
    tier: node
    app: flannel
    k8s-app: flannel
spec:
  template:
    spec:
      containers:
      - name: kube-flannel
        args:
        - --ip-masq
        - --kube-subnet-mgr
        - --iface=tun0
```

In [ ]:
cat << EOF > kube-flannel.yaml
---
kind: Namespace
apiVersion: v1
metadata:
  name: kube-flannel
  labels:
    k8s-app: flannel
    pod-security.kubernetes.io/enforce: privileged
---
kind: ClusterRole
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  labels:
    k8s-app: flannel
  name: flannel
rules:
- apiGroups:
  - ""
  resources:
  - pods
  verbs:
  - get
- apiGroups:
  - ""
  resources:
  - nodes
  verbs:
  - get
  - list
  - watch
- apiGroups:
  - ""
  resources:
  - nodes/status
  verbs:
  - patch
---
kind: ClusterRoleBinding
apiVersion: rbac.authorization.k8s.io/v1
metadata:
  labels:
    k8s-app: flannel
  name: flannel
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: flannel
subjects:
- kind: ServiceAccount
  name: flannel
  namespace: kube-flannel
---
apiVersion: v1
kind: ServiceAccount
metadata:
  labels:
    k8s-app: flannel
  name: flannel
  namespace: kube-flannel
---
kind: ConfigMap
apiVersion: v1
metadata:
  name: kube-flannel-cfg
  namespace: kube-flannel
  labels:
    tier: node
    k8s-app: flannel
    app: flannel
data:
  cni-conf.json: |
    {
      "name": "cbr0",
      "cniVersion": "0.3.1",
      "plugins": [
        {
          "type": "flannel",
          "delegate": {
            "hairpinMode": true,
            "isDefaultGateway": true
          }
        },
        {
          "type": "portmap",
          "capabilities": {
            "portMappings": true
          }
        }
      ]
    }
  net-conf.json: |
    {
      "Network": "10.244.0.0/16",
      "EnableNFTables": false,
      "Backend": {
        "Type": "vxlan"
      }
    }
---
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: kube-flannel-ds
  namespace: kube-flannel
  labels:
    tier: node
    app: flannel
    k8s-app: flannel
spec:
  selector:
    matchLabels:
      app: flannel
  template:
    metadata:
      labels:
        tier: node
        app: flannel
    spec:
      affinity:
        nodeAffinity:
          requiredDuringSchedulingIgnoredDuringExecution:
            nodeSelectorTerms:
            - matchExpressions:
              - key: kubernetes.io/os
                operator: In
                values:
                - linux
      hostNetwork: true
      priorityClassName: system-node-critical
      tolerations:
      - operator: Exists
        effect: NoSchedule
      serviceAccountName: flannel
      initContainers:
      - name: install-cni-plugin
        image: docker.io/flannel/flannel-cni-plugin:v1.4.0-flannel1
        command:
        - cp
        args:
        - -f
        - /flannel
        - /opt/cni/bin/flannel
        volumeMounts:
        - name: cni-plugin
          mountPath: /opt/cni/bin
      - name: install-cni
        image: docker.io/flannel/flannel:v0.25.1
        command:
        - cp
        args:
        - -f
        - /etc/kube-flannel/cni-conf.json
        - /etc/cni/net.d/10-flannel.conflist
        volumeMounts:
        - name: cni
          mountPath: /etc/cni/net.d
        - name: flannel-cfg
          mountPath: /etc/kube-flannel/
      containers:
      - name: kube-flannel
        image: docker.io/flannel/flannel:v0.25.1
        command:
        - /opt/bin/flanneld
        args:
        - --ip-masq
        - --kube-subnet-mgr
        - --iface=tun0
        resources:
          requests:
            cpu: "100m"
            memory: "50Mi"
        securityContext:
          privileged: false
          capabilities:
            add: ["NET_ADMIN", "NET_RAW"]
        env:
        - name: POD_NAME
          valueFrom:
            fieldRef:
              fieldPath: metadata.name
        - name: POD_NAMESPACE
          valueFrom:
            fieldRef:
              fieldPath: metadata.namespace
        - name: EVENT_QUEUE_DEPTH
          value: "5000"
        volumeMounts:
        - name: run
          mountPath: /run/flannel
        - name: flannel-cfg
          mountPath: /etc/kube-flannel/
        - name: xtables-lock
          mountPath: /run/xtables.lock
      volumes:
      - name: run
        hostPath:
          path: /run/flannel
      - name: cni-plugin
        hostPath:
          path: /opt/cni/bin
      - name: cni
        hostPath:
          path: /etc/cni/net.d
      - name: flannel-cfg
        configMap:
          name: kube-flannel-cfg
      - name: xtables-lock
        hostPath:
          path: /run/xtables.lock
          type: FileOrCreate
EOF

In [ ]:
kubectl apply -f kube-flannel.yaml

namespace/kube-flannel created
clusterrole.rbac.authorization.k8s.io/flannel created
clusterrolebinding.rbac.authorization.k8s.io/flannel created
serviceaccount/flannel created
configmap/kube-flannel-cfg created
daemonset.apps/kube-flannel-ds created


---
taints

In [ ]:
kubectl taint nodes --all node-role.kubernetes.io/control-plane-
kubectl taint nodes --all node.kubernetes.io/disk-pressure-
kubectl taint nodes --all node.kubernetes.io/not-ready-

In [ ]:
kubectl describe node free-vpn | grep -i8 taints

In [ ]:
kubectl get nodes -o wide

NAME         STATUS   ROLES           AGE     VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE                         KERNEL-VERSION                       CONTAINER-RUNTIME
free-vpn     Ready    control-plane   24m     v1.29.3   10.146.0.3    <none>        Debian GNU/Linux 11 (bullseye)   5.10.0-26-cloud-amd64                containerd://1.6.24
msd-npc112   Ready    <none>          7m24s   v1.29.3   10.8.0.2      <none>        Ubuntu 22.04.3 LTS               5.15.146.1-microsoft-standard-WSL2   containerd://1.7.2


---
on another worker node

In [ ]:
sudo apt-get update
sudo apt-get install containerd
sudo containerd --version

In [ ]:
sudo sysctl net.ipv4.ip_forward=1

In [ ]:
cat /proc/sys/net/ipv4/ip_forward

In [ ]:
sudo kubeadm join jhub.dtype.info:6443 --token secret.secretsecretsecr \
--discovery-token-ca-cert-hash sha256:secretsecretsecretsecretsecretsecretsecretsecretsecretsecretsecr

---
on control-plane node

In [ ]:
kubectl describe node msd-npc112 | grep -i2 taints

                    volumes.kubernetes.io/controller-managed-attach-detach: true
CreationTimestamp:  Sat, 13 Apr 2024 01:25:39 +0900
Taints:             <none>
Unschedulable:      false
Lease:


In [ ]:
kubectl describe node msd-npc112

In [ ]:
kubectl get nodes

NAME         STATUS   ROLES           AGE   VERSION
free-vpn     Ready    control-plane   28m   v1.29.3
msd-npc112   Ready    <none>          13m   v1.29.3


In [ ]:
kubectl get pods --all-namespaces

NAMESPACE     NAME                               READY   STATUS    RESTARTS   AGE
kube-system   coredns-76f75df574-nn6nb           1/1     Running   0          27m
kube-system   coredns-76f75df574-pz6k4           1/1     Running   0          27m
kube-system   etcd-free-vpn                      1/1     Running   8          27m
kube-system   kube-apiserver-free-vpn            1/1     Running   8          27m
kube-system   kube-controller-manager-free-vpn   1/1     Running   50         27m
kube-system   kube-proxy-6zj7x                   1/1     Running   0          12m
kube-system   kube-proxy-8sms9                   1/1     Running   0          27m
kube-system   kube-scheduler-free-vpn            1/1     Running   53         27m


---
[Dashboard by Helm](https://github.com/kubernetes/dashboard)

In [ ]:
helm uninstall kubernetes-dashboard -n kubernetes-dashboard

release "kubernetes-dashboard" uninstalled


In [ ]:
kubectl delete namespace kubernetes-dashboard

namespace "kubernetes-dashboard" deleted


In [ ]:
helm upgrade --install --repo https://kubernetes.github.io/dashboard/ kubernetes-dashboard kubernetes-dashboard \
--create-namespace --namespace kubernetes-dashboard \
--set web.containers.args[0]='--insecure-bind-address=0.0.0.0'

Release "kubernetes-dashboard" does not exist. Installing it now.
NAME: kubernetes-dashboard
LAST DEPLOYED: Thu Apr 18 10:04:55 2024
NAMESPACE: kubernetes-dashboard
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
*************************************************************************************************
*** PLEASE BE PATIENT: Kubernetes Dashboard may need a few minutes to get up and become ready ***
*************************************************************************************************

Congratulations! You have just installed Kubernetes Dashboard in your cluster.

To access Dashboard run:
  kubectl -n kubernetes-dashboard port-forward svc/kubernetes-dashboard-kong-proxy 8443:443

NOTE: In case port-forward command does not work, make sure that kong service name is correct.
      Check the services in Kubernetes Dashboard namespace using:
        kubectl -n kubernetes-dashboard get svc

Dashboard will be available at:
  https://localhost:8443


In [ ]:
kubectl get pods -n kubernetes-dashboard -o wide

In [ ]:
kubectl get svc -n kubernetes-dashboard -o wide

In [ ]:
export POD_NAME=$(
  kubectl get pods -n kubernetes-dashboard -l 'app=kubernetes-dashboard-kong' -o jsonpath='{.items[0].metadata.name}'
)

In [ ]:
kubectl port-forward $POD_NAME -n kubernetes-dashboard 8443:8443

Forwarding from 127.0.0.1:8443 -> 8443
Forwarding from [::1]:8443 -> 8443
Handling connection for 8443


In [ ]:
kubectl port-forward svc/kubernetes-dashboard-kong-proxy -n kubernetes-dashboard 8443:443

Forwarding from 127.0.0.1:8443 -> 8443
Forwarding from [::1]:8443 -> 8443
Handling connection for 8443
Handling connection for 8443


In [ ]:
echo 'Visit https://localhost:8443 to use your application'

In [ ]:
export POD_NAME=$(
  kubectl get pods -n kubernetes-dashboard -l 'app.kubernetes.io/name=kubernetes-dashboard-web' -o jsonpath='{.items[0].metadata.name}'
)

In [ ]:
kubectl port-forward $POD_NAME -n kubernetes-dashboard 8443:8443

In [ ]:
kubectl port-forward svc/kubernetes-dashboard-web -n kubernetes-dashboard 8443:8000

Forwarding from 127.0.0.1:8443 -> 8000
Forwarding from [::1]:8443 -> 8000
Handling connection for 8443
Handling connection for 8443


In [ ]:
echo 'Visit http://localhost:8443 to use your application'

---
[Dashboard by Manifest-based YAML](https://kubernetes.io/zh-cn/docs/tasks/access-application-cluster/web-ui-dashboard/)

In [ ]:
wget https://raw.githubusercontent.com/kubernetes/dashboard/v2.7.0/aio/deploy/recommended.yaml -O kube-dashboard.yaml

``` yaml
kind: Deployment
apiVersion: apps/v1
metadata:
  labels:
    k8s-app: dashboard-metrics-scraper
  name: dashboard-metrics-scraper
  namespace: kubernetes-dashboard
spec:
  template:
    metadata:
      labels:
        k8s-app: dashboard-metrics-scraper
      annotations:
        seccompProfile: 'runtime/default'
```

In [ ]:
kubectl apply -f kube-dashboard.yaml

namespace/kubernetes-dashboard unchanged
serviceaccount/kubernetes-dashboard unchanged
service/kubernetes-dashboard unchanged
secret/kubernetes-dashboard-certs unchanged
secret/kubernetes-dashboard-csrf configured
secret/kubernetes-dashboard-key-holder unchanged
configmap/kubernetes-dashboard-settings unchanged
role.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
clusterrole.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
rolebinding.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
clusterrolebinding.rbac.authorization.k8s.io/kubernetes-dashboard unchanged
deployment.apps/kubernetes-dashboard configured
service/dashboard-metrics-scraper unchanged
deployment.apps/dashboard-metrics-scraper configured


In [ ]:
kubectl port-forward svc/kubernetes-dashboard -n kubernetes-dashboard 8443:443

Forwarding from 127.0.0.1:8443 -> 8443
Forwarding from [::1]:8443 -> 8443


In [ ]:
echo 'Visit https://localhost:8443 to use your application'

---
service account

In [ ]:
kubectl create serviceaccount saintway -n kubernetes-dashboard

serviceaccount/saintway created


In [ ]:
kubectl create clusterrolebinding dashboard-admin-binding --clusterrole=cluster-admin --serviceaccount=kubernetes-dashboard:saintway

In [ ]:
kubectl create token saintway -n kubernetes-dashboard

---
ingress tls secret

In [ ]:
kubectl apply -f - << EOF
apiVersion: v1
kind: Secret
metadata:
  name: dashboard-tls
  namespace: kubernetes-dashboard
data:
  tls.crt: $(sudo base64 -w 0 /etc/letsencrypt/live/registry.dtype.info/fullchain.pem)
  tls.key: $(sudo base64 -w 0 /etc/letsencrypt/live/registry.dtype.info/privkey.pem)
type: kubernetes.io/tls
EOF

secret/dashboard-tls created


---
ingress

In [ ]:
kubectl apply -f - << EOF
apiVersion: traefik.containo.us/v1alpha1
kind: Middleware
metadata:
  name: prefix
  namespace: kubernetes-dashboard
spec:
  stripPrefix:
    forceSlash: false
    prefixes:
    - /kubernetes
EOF

In [ ]:
kubectl apply -f - << EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubernetes.io/tls-acme: 'true'
    traefik.ingress.kubernetes.io/router.middlewares: kubernetes-dashboard-prefix@kubernetes
  name: kubernetes-dashboard
  namespace: kubernetes-dashboard
spec:
  tls:
  - hosts:
    - registry.dtype.info
    secretName: dashboard-tls
  rules:
  - host: registry.dtype.info
    http:
      paths:
      - path: /kubernetes
        pathType: Prefix
        backend:
          service:
            name: kubernetes-dashboard-kong-proxy
            port:
              number: 443
EOF

ingress.networking.k8s.io/kubernetes-dashboard configured


In [ ]:
kubectl delete ingress kubernetes-dashboard -n kubernetes-dashboard

ingress.networking.k8s.io "kubernetes-dashboard" deleted


---
ingress

In [ ]:
cat << EOF | sudo tee /opt/tljh/state/rules/kubernetes.toml
[http.middlewares]
  [http.middlewares.prefix.stripPrefix]
    prefixes = ["/kubernetes"]
    forceSlash = false
EOF

In [ ]:
kubectl apply -f - << EOF
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  annotations:
    kubernetes.io/tls-acme: 'true'
    traefik.ingress.kubernetes.io/router.middlewares: prefix@file
  name: kubernetes-dashboard
  namespace: kubernetes-dashboard
spec:
  tls:
  - hosts:
    - registry.dtype.info
    secretName: dashboard-tls
  rules:
  - host: registry.dtype.info
    http:
      paths:
      - path: /kubernetes
        pathType: Prefix
        backend:
          service:
            name: kubernetes-dashboard-web
            port:
              number: 8000
EOF

ingress.networking.k8s.io/kubernetes-dashboard created


---
shutdown

In [ ]:
sudo systemctl stop kubelet

In [ ]:
sudo systemctl disable kubelet